In [1]:
import numpy as np
import matplotlib as plt
import nfl_data_py as nfl
import pandas as pd

In [2]:
year = 2020
nfl_2020 = nfl.import_pbp_data([year],  downcast=True, cache=False, alt_path=None)

2020 done.
Downcasting floats.


In [3]:
columns = nfl_2020.columns

In [4]:
[print(x) for x in columns]

play_id
game_id
old_game_id
home_team
away_team
season_type
week
posteam
posteam_type
defteam
side_of_field
yardline_100
game_date
quarter_seconds_remaining
half_seconds_remaining
game_seconds_remaining
game_half
quarter_end
drive
sp
qtr
down
goal_to_go
time
yrdln
ydstogo
ydsnet
desc
play_type
yards_gained
shotgun
no_huddle
qb_dropback
qb_kneel
qb_spike
qb_scramble
pass_length
pass_location
air_yards
yards_after_catch
run_location
run_gap
field_goal_result
kick_distance
extra_point_result
two_point_conv_result
home_timeouts_remaining
away_timeouts_remaining
timeout
timeout_team
td_team
td_player_name
td_player_id
posteam_timeouts_remaining
defteam_timeouts_remaining
total_home_score
total_away_score
posteam_score
defteam_score
score_differential
posteam_score_post
defteam_score_post
score_differential_post
no_score_prob
opp_fg_prob
opp_safety_prob
opp_td_prob
fg_prob
safety_prob
td_prob
extra_point_prob
two_point_conversion_prob
ep
epa
total_home_epa
total_away_epa
total_home_rush_epa


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [15]:
#[print(x) for x in columns if 'tac' in x]

In [5]:
team_ids = np.unique( nfl_2020['home_team'].to_list()  )

In [6]:
team_ids

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV',
       'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF',
       'TB', 'TEN', 'WAS'], dtype='<U3')

In [7]:
assert(len(team_ids)) == 32, 'Post expansion'

In [8]:
seattle_home = nfl_2020[ nfl_2020['home_team'] == 'SEA' ]

seattle_away = nfl_2020[ nfl_2020['away_team'] == 'SEA' ]

we want the tackles on seattle, so seattle is the team which possesses the ball

In [17]:
seattle_home = seattle_home[seattle_home['possession_team'] == 'SEA']

#[print(x) for x in seattle_home]
opposition_seattle_home = seattle_home['away_team']

#seattle_away = seattle_away[seattle_away['possession_team'] == 'SEA']
#opposition_seattle_away = seattle_away['home_team']

AttributeError: 'Series' object has no attribute 'columns'

**when seattle is at home - tackles**

In [13]:
solo = len(opposition_seattle_home[ opposition_seattle_home['solo_tackle'] ==1 ] )
assisted = len(opposition_seattle_home[ opposition_seattle_home['assist_tackle'] ==1 ] )
seattle_tackles_at_home = solo + assisted

KeyError: 'solo_tackle'

In [12]:
seattle_tackles_at_home

NameError: name 'seattle_tackles_at_home' is not defined

**away tackles**

In [23]:
solo = len(seattle_away[ seattle_away['solo_tackle'] ==1 ] )
assisted = len(seattle_away[ seattle_away['assist_tackle'] ==1 ] )
seattle_tackles_away = solo + assisted

In [24]:
seattle_tackles_away

444

:-) less tackles away!!! okay generalize it

In [19]:
def filt_by_team(team, nfl_2020):
    #assert type(team) == str, 'something went wrong with team ' + str(team) + ' which is of team ' + str(type(team))
    home = nfl_2020[ nfl_2020['home_team'] == str(team) ]
    away = nfl_2020[ nfl_2020['away_team'] ==  str(team) ]
    
    home = home[home['possession_team'] != str(team)]
    away = away[away['possession_team'] != str(team)]
    
    return home, away

def total_tackles_home( home ):
    solo = len(home[ home['solo_tackle'] == 1 ] )
    assisted = len(home[ home['assist_tackle'] == 1 ] )
    tackles_at_home = solo + assisted
    return tackles_at_home
    
def total_tackles_away( away ):
    solo = len(away[ away['solo_tackle'] ==1 ] )
    assisted = len(away[away['assist_tackle'] ==1 ] )
    tackles_away = solo + assisted
    return tackles_away


def completions( home ):
    return len(home[ home['complete_pass'] == 1 ] )


Make dictionary for data where each key is a different team

In [49]:
data = {}
for team in team_ids:
    data[str(team)] = np.zeros((7))

In [50]:
for team in team_ids:
    home, away = filt_by_team(team, nfl_2020)
    
    home_tackles, away_tackles = total_tackles_home( home ), total_tackles_away( away )
    data[team][3], data[team][4] = home_tackles, away_tackles
    '''completions'''
    #channel 
    data[team][5], data[team][6] = completions( home ), completions( away )
  
    

In [39]:
data.keys()

dict_keys(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL', 'DEN', 'DET', 'GB', 'HOU', 'IND', 'JAX', 'KC', 'LA', 'LAC', 'LV', 'MIA', 'MIN', 'NE', 'NO', 'NYG', 'NYJ', 'PHI', 'PIT', 'SEA', 'SF', 'TB', 'TEN', 'WAS'])

In [52]:
# this is done tediously by hand because no data availabe in a nice format
# I guess costume color is not a big priority for football statisticians

data['ARI'][0:3] = (151,35,63)
data['ATL'][0:3] = (167,25,48)
data['BAL'][0:3] = (26,25,95)
data['BUF'][0:3] = (0,51,141)
data['CAR'][0:3] = (0,133,202)
data['CHI'][0:3] = (11,22,42)
data['CIN'][0:3] = (251,79,20)
data['CLE'][0:3] = (49,29,0)
data['DAL'][0:3] = (0,53,148)
data['DEN'][0:3] = (251,79,20)
data['DET'][0:3] = (0,118,182)
data['GB'][0:3] = (24,48,40)
data['HOU'][0:3] = (3,32,47)
data['IND'][0:3] = (0,44,95)
data['JAX'][0:3] = (16,24,32)
data['KC'][0:3] = (227,24,55)
data['LA'][0:3] = (0,53,148)
data['LAC'][0:3] = (0,128,198)
data['LV'][0:3] = (0,0,0)
data['MIA'][0:3] = (0,142,151)
data['MIN'][0:3] = (79,38,131)
data['NE'][0:3] = (0,34,68)
data['NO'][0:3] = (211,188,141)
data['NYG'][0:3] = (1,35,82)
data['NYJ'][0:3] = (18,87,64)
data['PHI'][0:3] = (0,76,84)
data['PIT'][0:3] = (255,182,18)
data['SEA'][0:3] = (0,34,68)
data['SF'][0:3] = (170,0,0)
data['TB'][0:3] = (213,10,10)
data['TEN'][0:3] = (12,35,64)
data['WAS'][0:3] = (90,20,20)

Now let's save this data in a python dict format,

In [53]:
data['ARI']

array([151.,  35.,  63., 427., 423.,   0.,   0.])

In [54]:
import pickle

# save dictionary to nfl2020_data.pkl file
with open('nfl2020_data.pkl', 'wb') as fp:
    pickle.dump(data, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [55]:
with open('nfl2020_data.pkl', 'rb') as fp:
    data_loaded = pickle.load(fp)
    print('nfl data incoming...')
    print(data_loaded)

nfl data incoming...
{'ARI': array([151.,  35.,  63., 427., 423.,   0.,   0.]), 'ATL': array([167.,  25.,  48., 407., 405.,   0.,   0.]), 'BAL': array([ 26.,  25.,  95., 420., 470.,   0.,   0.]), 'BUF': array([  0.,  51., 141., 537., 465.,   0.,   0.]), 'CAR': array([  0., 133., 202., 409., 403.,   0.,   0.]), 'CHI': array([ 11.,  22.,  42., 425., 474.,   0.,   0.]), 'CIN': array([251.,  79.,  20., 412., 399.,   0.,   0.]), 'CLE': array([ 49.,  29.,   0., 409., 522.,   0.,   0.]), 'DAL': array([  0.,  53., 148., 487., 422.,   0.,   0.]), 'DEN': array([251.,  79.,  20., 412., 424.,   0.,   0.]), 'DET': array([  0., 118., 182., 428., 419.,   0.,   0.]), 'GB': array([ 24.,  48.,  40., 465., 410.,   0.,   0.]), 'HOU': array([  3.,  32.,  47., 437., 441.,   0.,   0.]), 'IND': array([  0.,  44.,  95., 416., 448.,   0.,   0.]), 'JAX': array([ 16.,  24.,  32., 428., 432.,   0.,   0.]), 'KC': array([227.,  24.,  55., 526., 440.,   0.,   0.]), 'LA': array([  0.,  53., 148., 424., 482.,   0.,   0

In [43]:
data_loaded['ARI']

array([  0., 427., 423.,   0.,   0.])

In [ ]:
def filt_by_team_compl(team, nfl_2020):
    home = nfl_2020[ nfl_2020['home_team'] == str(team) ]
    away = nfl_2020[ nfl_2020['away_team'] ==  str(team) ]
    
    home = home[home['possession_team'] == str(team)]
    away = away[away['possession_team'] == str(team)]

    return home, away


def completions( home ):
    return len(home[ home['complete_pass'] == 1 ] )
    



In [ ]:
data_compl = {}
for team in team_ids:
    data_compl[str(team)] = np.zeros((2))

In [ ]:

for team in team_ids:
    home, away = filt_by_team_compl(team, nfl_2020)
    
    home_compl, away_compl = completions( home ), completions( away )
    
    data_compl[team][0], data_compl[team][1] = home_compl, away_compl
    